In [1]:
from huggingface_hub import login, HfFolder
import os
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')

login(token=huggingface_token, add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\Zhenya\.cache\huggingface\token
Login successful


In [2]:
import logging
import os
import re
import shutil
import sys
import time
from dataclasses import dataclass, field
from functools import partial
from pathlib import Path
from typing import Any, Dict, List, Optional, Union

import datasets
import evaluate
import numpy as np
import torch
import torch.nn as nn
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import (
    DatasetDict,
    IterableDataset,
    IterableDatasetDict,
    concatenate_datasets,
    interleave_datasets,
    load_dataset,
)
from huggingface_hub import create_repo, get_full_repo_name, upload_folder
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AddedToken,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    WhisperConfig,
    WhisperFeatureExtractor,
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperTokenizerFast,
    get_scheduler
)
from transformers.modeling_outputs import BaseModelOutput
from transformers.models.whisper.english_normalizer import BasicTextNormalizer, EnglishTextNormalizer
from transformers.utils import check_min_version
from transformers.utils.versions import require_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.34.0.dev0")

require_version("datasets>=2.14.6", "To fix: `pip install --upgrade datasets`")

In [3]:
from run_distillation import (ModelArguments, DataTrainingArguments, DistillationTrainingArguments, DataCollatorSpeechSeq2SeqWithPadding,
                               log_metric, log_pred, get_parameter_names, concatenate_datasets, rotate_checkpoints)

In [4]:
%load_ext autoreload
%autoreload 1

%aimport src.utils
%aimport src.load_model

from src.utils import prepare_accelerator
from src.load_model import load_config_feature_ext_tokenizer, load_processor, load_whisper_model

In [5]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, DistillationTrainingArguments))
model_args, data_args, training_args = parser.parse_json_file(json_file="pipe_configs/distiling_v0.json")

In [6]:
accelerator, model_dtype, logger = prepare_accelerator(input_dtype=training_args.dtype, project_name=data_args.wandb_project, training_args=training_args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zekamrozek. Use `wandb login --relogin` to force relogin


05/08/2024 19:07:54 - WARNING - src.utils - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [7]:
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

In [8]:
last_checkpoint = None

In [9]:
os.makedirs(training_args.output_dir, exist_ok=True)

In [10]:
def load_mozila_labaled_dataset(path, split, text_name, DEBUG_MODE=False):

    loaded_dataset = datasets.load_from_disk(f'{path}/{split}')
    if DEBUG_MODE:
       loaded_dataset = loaded_dataset.select(range(100))
    
    if text_name != "text":
        loaded_dataset = loaded_dataset.rename_column(text_name, "text")

    return loaded_dataset
    

In [11]:
DEBUG_MODE = False
raw_datasets = DatasetDict()
train_datasets_list = []

mozila_pseudo_labeled_path = 'datasets/labeled_mozila_3'

for split in ['train', 'validation']:
    loaded_dataset = load_mozila_labaled_dataset(mozila_pseudo_labeled_path, split, 'sentence', DEBUG_MODE)
    train_datasets_list.append(loaded_dataset)

raw_datasets["train"] = concatenate_datasets(train_datasets_list)
raw_datasets["eval"] = load_mozila_labaled_dataset(mozila_pseudo_labeled_path, 'test', 'sentence', DEBUG_MODE)


In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'text', 'condition_on_prev', 'whisper_transcript'],
        num_rows: 5187
    })
    eval: Dataset({
        features: ['path', 'audio', 'text', 'condition_on_prev', 'whisper_transcript'],
        num_rows: 1734
    })
})

In [13]:
# 7. Load pretrained model, tokenizer, and feature extractor
config = WhisperConfig.from_pretrained(
    (model_args.config_name if model_args.config_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    token=model_args.token,
)
feature_extractor = WhisperFeatureExtractor.from_pretrained(
    (model_args.feature_extractor_name if model_args.feature_extractor_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    token=model_args.token,
)
tokenizer = WhisperTokenizerFast.from_pretrained(
    (model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    token=model_args.token,
)

# override timestamp tokens until tokenizer issues are fixed in transformers
timestamps = [AddedToken("<|%.2f|>" % (i * 0.02), lstrip=False, rstrip=False) for i in range(1500 + 1)]
tokenizer.add_tokens(timestamps)

# The teacher model can safely be cast to the dtype of training since we don't
# update the params
teacher_model = WhisperForConditionalGeneration.from_pretrained(
    model_args.teacher_model_name_or_path,
    cache_dir=model_args.cache_dir,
    token=model_args.token,
    low_cpu_mem_usage=True,
    torch_dtype=model_dtype,
    attn_implementation=model_args.attn_implementation,
)

student_model = WhisperForConditionalGeneration.from_pretrained(
    model_args.model_name_or_path,
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    subfolder=model_args.subfolder,
    token=model_args.token,
    low_cpu_mem_usage=True,
    attn_implementation=model_args.attn_implementation,
)

loading configuration file ./models/student_model\config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-medium",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 24,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_f

In [14]:
if student_model.config.decoder_start_token_id is None or teacher_model.config.decoder_start_token_id is None:
    raise ValueError(
        f"Make sure that `config.decoder_start_token_id` is correctly defined for both the "
        f"student and teacher model. Got {student_model.config.decoder_start_token_id} for the "
        f"student and {teacher_model.config.decoder_start_token_id} for the teacher."
    )

In [15]:
if training_args.gradient_checkpointing:
    student_model.gradient_checkpointing_enable()

def set_trainable_parameters(module, requires_grad=False):
    for param in module.parameters():
        param.requires_grad = requires_grad
    module._requires_grad = requires_grad

# freeze student encoder if necessary
if training_args.freeze_encoder:
    set_trainable_parameters(student_model.model.encoder, requires_grad=False)
    student_model.model.encoder.gradient_checkpointing = False

if training_args.freeze_decoder:
    set_trainable_parameters(student_model.model.decoder, requires_grad=False)
    student_model.model.decoder.gradient_checkpointing = False
    # un-freeze LM head parameters (and consequently word embeddings), frozen when frozing decoder since tied word embedding and LM head
    set_trainable_parameters(student_model.proj_out, requires_grad=True) 
    

if training_args.freeze_embed_positions:
    # set_trainable_parameters(student_model.model.decoder.embed_tokens, requires_grad=False)
    set_trainable_parameters(student_model.model.decoder.embed_positions, requires_grad=False)
    if student_model.model.decoder.gradient_checkpointing:
        logger.info(
            "Disabling gradient checkpointing in the decoder since it's incompatible with `freeze_embed_positions`."
        )

logger.info(
    f"Number of trainable parameters: {sum(p.numel() for p in student_model.parameters() if p.requires_grad):.3e}"
)

05/08/2024 19:08:04 - INFO - src.utils - Disabling gradient checkpointing in the decoder since it's incompatible with `freeze_embed_positions`.
05/08/2024 19:08:04 - INFO - src.utils - Number of trainable parameters: 8.670e+07


In [16]:
share_hidden_states = training_args.freeze_encoder and student_model.config.d_model == teacher_model.config.d_model
if share_hidden_states:
    # tie the weights for the teacher encoder if we're freezing the student and it's the same as the teacher
    teacher_model.model.encoder = student_model.model.encoder    

In [17]:
raw_datasets['train'][0]

{'path': 'mozila-uk-0',
 'audio': {'path': None,
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00, -3.05175781e-05, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'Котра корова богато риче, мало молока дає. Не жди ніколи слушної пори – твоя мовчанка може стать ганьбою! Це дуже потужний антропогенний вплив на природу. Виявлялось - вiн не плакав, а тiльки витирав свої хворi очi!.. Є два способи командувати жінкою, але їх ніхто не знає Якщо можна я відповім.',
 'condition_on_prev': None,
 'whisper_transcript': '<|startoftranscript|><|uk|><|transcribe|><|0.00|> Которе корово багато речі, мало молока дає.<|5.00|><|5.00|> Нежиди ніколи слушною пори твоя мовчанка може статігать бою.<|11.00|><|11.00|> Це дуже потужний антропогенний вплив на природу.<|15.00|><|15.00|> Виявлялось він не плакав, а тільки витирав свої хворі очі.<|20.00|><|20.00|> Є два способи командувати жінкою, але їх ніхто не знає.<|24.00|><|24.00|> Якщо можна, я відпові

In [18]:
if hasattr(teacher_model.generation_config, "is_multilingual") and teacher_model.generation_config.is_multilingual:
    # We need to set the language and task ids for previously multilingual checkpoints
    is_multilingual = True
    tokenizer.set_prefix_tokens(language=data_args.language, task=data_args.task, predict_timestamps=False)
    student_model.generation_config.update(
        **{
            "language": data_args.language,
            "task": data_args.task,
        }
    )
    print("is_multilingual", data_args.language)

is_multilingual uk


In [19]:
# 8. Create a single speech processor - make sure all processes wait until data is saved
if accelerator.is_main_process:
    feature_extractor.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    # save the config and generation config as well
    config.save_pretrained(training_args.output_dir)
    student_model.generation_config.save_pretrained(training_args.output_dir)

accelerator.wait_for_everyone()
processor = WhisperProcessor.from_pretrained(training_args.output_dir)

Feature extractor saved in ./models/distiled_student_model_moz_v1\preprocessor_config.json
tokenizer config file saved in ./models/distiled_student_model_moz_v1\tokenizer_config.json
Special tokens file saved in ./models/distiled_student_model_moz_v1\special_tokens_map.json
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 2167

In [20]:
# 9. Resample speech dataset: `datasets` takes care of automatically loading and resampling the audio,
# so we just need to set the correct target sampling rate.
sampling_rate = feature_extractor.sampling_rate
raw_datasets = raw_datasets.cast_column(
    data_args.audio_column_name,
    datasets.features.Audio(sampling_rate=sampling_rate),
)

# 10. Preprocessing the datasets: we need to read the audio files as arrays and tokenize the targets.
# 10.1: Define the pre-processing constants
max_input_length = int(data_args.max_duration_in_seconds * sampling_rate)
min_input_length = int(data_args.min_duration_in_seconds * sampling_rate)
max_label_length = (
    data_args.max_label_length if data_args.max_label_length is not None else student_model.config.max_length
)

timestamp_probability = data_args.timestamp_probability
condition_on_prev_probability = data_args.condition_on_prev_probability
return_timestamps = data_args.return_timestamps if timestamp_probability > 0 else False

timestamp_ids = tokenizer.timestamp_ids()
timestamp_begin = tokenizer.all_special_ids[-1]
timestamp_position = 3 if is_multilingual else 1

decoder_start_token_id = student_model.config.decoder_start_token_id  # <|startoftranscript|>
decoder_prev_token_id = tokenizer.all_special_ids[-3]  # <|startofprev|>
prompt_cutoff_length = max_label_length // 2

num_workers = data_args.preprocessing_num_workers
dataloader_num_workers = training_args.dataloader_num_workers
prefetch_factor = training_args.dataloader_prefetch_factor

In [21]:
metric = evaluate.load("wer")
normalizer = (
    BasicTextNormalizer()
    if data_args.language is not None
    else EnglishTextNormalizer(tokenizer.english_spelling_normalizer)
)
wer_threshold = data_args.wer_threshold
use_pseudo_labels = data_args.use_pseudo_labels
train_text_column_name = "whisper_transcript" if use_pseudo_labels else "text"

In [22]:
# 10.3: filter training data based on WER threshold -> this is KEY to good distillation performance
def is_wer_in_range(ground_truth, whisper_transcript):
    norm_ground_truth = normalizer(ground_truth)
    if whisper_transcript is not None and whisper_transcript.upper() == whisper_transcript:
        # filter entirely upper-case transcriptions: these are erroneous generations from large-v3
        return False
    elif len(norm_ground_truth) > 0 and whisper_transcript is not None:
        norm_whisper_transcript = normalizer(whisper_transcript)
        wer = 100 * metric.compute(predictions=[norm_whisper_transcript], references=[norm_ground_truth])
        return wer < wer_threshold
    else:
        # filter automatically since we can't know the WER
        return False

filter_by_wer_threshold = partial(
    raw_datasets["train"].filter,
    function=is_wer_in_range,
    input_columns=["text", "whisper_transcript"],
)

if wer_threshold is not None and use_pseudo_labels:
    with accelerator.main_process_first():
        raw_datasets["train"] = (
            filter_by_wer_threshold(num_proc=1, desc="filtering train dataset by wer")
            if not data_args.streaming
            else filter_by_wer_threshold()
        )

filtering train dataset by wer:   0%|          | 0/5187 [00:00<?, ? examples/s]

In [23]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'text', 'condition_on_prev', 'whisper_transcript'],
        num_rows: 3686
    })
    eval: Dataset({
        features: ['path', 'audio', 'text', 'condition_on_prev', 'whisper_transcript'],
        num_rows: 1734
    })
})

In [24]:
def prepare_train_dataset(batch):
    """
    Pre-process the raw dataset in a three stage process:
        1. Convert the audio arrays to log-mel spectrogram inputs
        2. Possibly filter the timestamp tokens from the token ids (depending on the timestamp probability)
        3. Possibly add prompt tokens if conditioning on previous text (depending on the conditioning probability)
    """
    # process audio input
    audio = [sample["array"] for sample in batch["audio"]]
    inputs = feature_extractor(audio, sampling_rate=sampling_rate)
    batch["input_features"] = inputs.input_features
    batch["input_length"] = [len(sample) for sample in audio]

    # process text targets - for training these are the Whisper-generated pseudo-labels
    input_str_batched = batch[train_text_column_name]
    condition_on_prev_batched = batch.get("condition_on_prev", len(input_str_batched) * [None])

    all_token_ids = []
    all_token_ids_unprompted = []
    for prev_ids, input_str in zip(condition_on_prev_batched, input_str_batched):
        token_ids = tokenizer(input_str, add_special_tokens=not use_pseudo_labels).input_ids

        # check whether we have timestamps in the PLs and filter if required
        has_timestamps = len(set(token_ids) & set(timestamp_ids)) > 0
        if has_timestamps:
            # sample from binomial distribution to get probability of training on timestamps
            predict_timestamps = bool(np.random.binomial(1, timestamp_probability))
            if not predict_timestamps:
                # filter timestamps and insert the <|notimestamps|> task token
                token_ids = [token for token in token_ids if token < timestamp_begin]
                token_ids.insert(timestamp_position, timestamp_begin)

        all_token_ids_unprompted.append(token_ids)
        # check whether to condition on previous text - we do this with probability condition_on_prev_probability
        condition_on_prev = bool(np.random.binomial(1, condition_on_prev_probability))
        if not condition_on_prev:
            prev_ids = None
        elif "condition_on_prev" not in batch and len(all_token_ids_unprompted) > 1:
            # prompt ids are the penultimate token ids in the batch
            prev_ids = all_token_ids_unprompted[-2]

        if prev_ids is not None:
            if has_timestamps and not predict_timestamps:
                # filter timestamp ids from prompt when not predicting timestamps
                prev_ids = [token for token in prev_ids if token < timestamp_begin]

            # check that the length of the prompt does not exceed more than half the max label length (224)
            if len(prev_ids) > prompt_cutoff_length:
                prev_ids = prev_ids[-prompt_cutoff_length + 1 :]
                prev_ids = [decoder_prev_token_id] + prev_ids

            # and that the total length of the labels does not exceed the max label length (448)
            if len(prev_ids + token_ids) > max_label_length:
                trim_length = len(prev_ids + token_ids) - max_label_length + 1
                prev_ids = prev_ids[trim_length:]
                prev_ids = [decoder_prev_token_id] + prev_ids

            token_ids = prev_ids + token_ids

        all_token_ids.append(token_ids)

    batch["labels"] = all_token_ids
    return batch

def prepare_eval_dataset(batch):
    # process audio input
    sample = batch["audio"]
    inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
    batch["input_features"] = inputs.input_features[0]
    batch["input_length"] = len(sample["array"])

    # process targets - for evaluation these are the ground-truth transcriptions
    input_str = batch["text"]
    batch["labels"] = tokenizer(input_str).input_ids
    return batch

In [25]:
all_eval_splits = ['eval']
raw_datasets_train_features = list(raw_datasets["train"].features.keys())
raw_datasets_train_features

['path', 'audio', 'text', 'condition_on_prev', 'whisper_transcript']

In [26]:
vectorized_datasets = IterableDatasetDict() if data_args.streaming else DatasetDict()
if training_args.do_train:
    # with streaming mode we can only have 1 worker, whereas with non-streaming
    # we can use `num_workers` (which is much faster)
    # We gate the pre-processing function accordingly
    map_fn_train = partial(
        raw_datasets["train"].map,
        function=prepare_train_dataset,
        remove_columns=raw_datasets_train_features,
        batched=True,
        batch_size=data_args.preprocessing_batch_size,
    )
    with accelerator.main_process_first():
        vectorized_datasets["train"] = (
            map_fn_train(num_proc=1, desc="preprocess train dataset")
            if not data_args.streaming
            else map_fn_train()
        )
if training_args.do_eval:
    for eval_split in all_eval_splits:
        raw_datasets_eval_features = list(raw_datasets[eval_split].features.keys())
        map_fn_eval = partial(
            raw_datasets[eval_split].map, function=prepare_eval_dataset, remove_columns=raw_datasets_eval_features
        )
        with accelerator.main_process_first():
            vectorized_datasets[eval_split] = (
                map_fn_eval(num_proc=1, desc="preprocess eval dataset")
                if not data_args.streaming
                else map_fn_eval()
            )

preprocess train dataset:   0%|          | 0/3686 [00:00<?, ? examples/s]

preprocess eval dataset:   0%|          | 0/1734 [00:00<?, ? examples/s]

In [27]:
vectorized_datasets['train']#[0]

Dataset({
    features: ['input_features', 'input_length', 'labels'],
    num_rows: 3686
})

In [28]:
# 10.5: Filter training data with inputs longer than `max_input_length`
def is_audio_in_length_range(length):
    return min_input_length < length < max_input_length

filter_by_audio_fn = partial(
    vectorized_datasets.filter, function=is_audio_in_length_range, input_columns=["input_length"]
)
with accelerator.main_process_first():
    vectorized_datasets = (
        filter_by_audio_fn(num_proc=1, desc="filtering train dataset by audio length")
        if not data_args.streaming
        else filter_by_audio_fn()
    )

# 10.6: Filter training data with labels longer than `max_label_length`
def is_labels_in_length_range(labels):
    return 0 < len(labels) <= max_label_length

filter_by_labels_fn = partial(
    vectorized_datasets.filter, function=is_labels_in_length_range, input_columns=["labels"]
)
with accelerator.main_process_first():
    vectorized_datasets = (
        filter_by_labels_fn(num_proc=1, desc="filtering train dataset")
        if not data_args.streaming
        else filter_by_labels_fn()
    )

filtering train dataset by audio length:   0%|          | 0/3686 [00:00<?, ? examples/s]

filtering train dataset by audio length:   0%|          | 0/1734 [00:00<?, ? examples/s]

filtering train dataset:   0%|          | 0/3686 [00:00<?, ? examples/s]

filtering train dataset:   0%|          | 0/1734 [00:00<?, ? examples/s]

In [29]:
# 11. Define Evaluation Metrics
def compute_metrics(preds, labels):
    # replace padded labels by the padding token
    for idx in range(len(labels)):
        labels[idx][labels[idx] == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True, decode_with_timestamps=return_timestamps)
    # we do not want to group tokens when computing the metrics
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # normalize everything and re-compute the WER
    norm_pred_str = [normalizer(pred) for pred in pred_str]
    norm_label_str = [normalizer(label) for label in label_str]
    # for logging, we need the pred/labels to match the norm_pred/norm_labels, so discard any filtered samples here
    pred_str = [pred_str[i] for i in range(len(norm_pred_str)) if len(norm_label_str[i]) > 0]
    label_str = [label_str[i] for i in range(len(norm_label_str)) if len(norm_label_str[i]) > 0]
    # filtering step to only evaluate the samples that correspond to non-zero normalized references:
    norm_pred_str = [norm_pred_str[i] for i in range(len(norm_pred_str)) if len(norm_label_str[i]) > 0]
    norm_label_str = [norm_label_str[i] for i in range(len(norm_label_str)) if len(norm_label_str[i]) > 0]

    wer = 100 * metric.compute(predictions=norm_pred_str, references=norm_label_str)
    return {"wer": wer, "wer_ortho": wer_ortho}, pred_str, label_str, norm_pred_str, norm_label_str

# 12. Define Training Schedule
# Store some constants
per_device_train_batch_size = int(training_args.per_device_train_batch_size)
train_batch_size = per_device_train_batch_size * accelerator.num_processes
gradient_accumulation_steps = int(training_args.gradient_accumulation_steps)
per_device_eval_batch_size = int(training_args.per_device_eval_batch_size)

if not data_args.streaming and training_args.max_steps < 0:
    num_epochs = int(training_args.num_train_epochs)
    steps_per_epoch = len(vectorized_datasets["train"]) // (train_batch_size * gradient_accumulation_steps)
    total_train_steps = steps_per_epoch * num_epochs
elif training_args.max_steps > 0:
    logger.info("max_steps is given, it will override any value given in num_train_epochs")
    total_train_steps = int(training_args.max_steps)
    if not data_args.streaming:
        steps_per_epoch = len(vectorized_datasets["train"]) // (train_batch_size * gradient_accumulation_steps)
        num_epochs = int(np.ceil(total_train_steps / steps_per_epoch))
    else:
        # Setting a very large number of epochs so we go as many times as necessary over the iterator.
        num_epochs = sys.maxsize
        steps_per_epoch = total_train_steps
else:
    raise ValueError("max_steps must be specified when training with a streaming (iterable) dataset")

if training_args.eval_steps is None:
    logger.info(
        f"eval_steps is not set, evaluating at the end of {'each epoch' if not data_args.streaming else 'training'}"
    )
    eval_steps = steps_per_epoch
else:
    eval_steps = training_args.eval_steps

05/08/2024 19:09:56 - INFO - src.utils - max_steps is given, it will override any value given in num_train_epochs


In [30]:
forbidden_module = [
    module
    for module, flag in [
        (student_model.model.encoder, training_args.freeze_encoder),
        (student_model.model.decoder, training_args.freeze_decoder)
    ]
    if flag
] or None

decay_parameters = get_parameter_names(
    student_model,
    [nn.LayerNorm],
    forbidden_module=forbidden_module,
)
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [param for name, param in student_model.named_parameters() if name in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [param for name, param in student_model.named_parameters() if name not in decay_parameters],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(
    params=optimizer_grouped_parameters,
    lr=training_args.learning_rate,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
)

# LR scheduler gets stepped by `num_processes` each time -> account for this in warmup / total steps
lr_scheduler = get_scheduler(
    name=training_args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps * accelerator.num_processes,
    num_training_steps=total_train_steps * accelerator.num_processes,
)

In [31]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=decoder_start_token_id,
    decoder_prev_token_id=decoder_prev_token_id,
    input_padding="longest",
    target_padding="max_length",
    max_target_length=max_label_length,
)

# 14. Define generation arguments - we need to do this before we wrap the models in DDP
# so that we can still access the configs
num_beams = (
    training_args.generation_num_beams
    if training_args.generation_num_beams is not None
    else getattr(student_model.generation_config, "num_beams", 1)
)

gen_kwargs = {
    "max_length": max_label_length,
    "num_beams": num_beams,
    "return_timestamps": return_timestamps,
}
if is_multilingual:
    # forcing the language and task tokens helps multilingual models in their generations
    gen_kwargs.update(
        {
            "language": data_args.language,
            "task": data_args.task,
        }
    )

In [32]:
gen_kwargs

{'max_length': 448,
 'num_beams': 1,
 'return_timestamps': False,
 'language': 'uk',
 'task': 'transcribe'}

In [33]:
student_model, teacher_model, optimizer, lr_scheduler = accelerator.prepare(
    student_model, teacher_model, optimizer, lr_scheduler
)

In [34]:
def kl_divergence(target_distribution, log_predicted_distribution, labels):
    kl_loss = nn.KLDivLoss(reduction="none")
    divergence = kl_loss(log_predicted_distribution, target_distribution)
    # ignore padded tokens from divergence, i.e. where labels are not set to -100
    padding_mask = labels >= 0
    padding_mask = padding_mask.unsqueeze(-1)
    divergence = divergence * padding_mask
    # take the average over the mini-batch
    divergence = divergence.sum() / padding_mask.sum()
    return divergence

# Define gradient update step fn
def train_step(
    batch,
    temperature=2.0,
):
    student_model.train()
    teacher_model.eval()

    student_outputs = student_model(**batch)
    with torch.no_grad():
        if share_hidden_states:
            # if the student and teacher share the same frozen encoder then we don't have to recompute the
            # encoder hidden-states for the teacher model, we can just re-use from the student
            encoder_outputs = BaseModelOutput(student_outputs.encoder_last_hidden_state.to(dtype=model_dtype))
            teacher_outputs = teacher_model(encoder_outputs=encoder_outputs, labels=batch["labels"])
        else:
            # do the full forward pass for the teacher model (encoder + decoder)
            teacher_outputs = teacher_model(**batch)

    # CE (data) loss
    ce_loss = student_outputs.loss
    # rescale distribution by temperature to ensure gradients scale correctly
    teacher_distribution = nn.functional.softmax(teacher_outputs.logits / temperature, dim=-1)
    # log softmax of student predictions for numerical stability
    student_distribution = nn.functional.log_softmax(student_outputs.logits / temperature, dim=-1)
    # KL-divergence loss (scaled by temperature)
    kl_loss = kl_divergence(teacher_distribution, student_distribution, batch["labels"]) * temperature**2

    # use Distil-Whisper formulation (fix weight of CE loss and tune KL weight)
    loss = 0.8 * ce_loss + training_args.kl_weight * kl_loss
    metrics = {"loss": loss, "ce_loss": ce_loss, "kl_loss": kl_loss}
    return loss, metrics

# Define eval fn
def eval_step(batch):
    student_model.eval()
    teacher_model.eval()

    with torch.no_grad():
        student_outputs = student_model(**batch)
        if share_hidden_states:
            encoder_outputs = BaseModelOutput(student_outputs.encoder_last_hidden_state.to(dtype=model_dtype))
            teacher_outputs = teacher_model(encoder_outputs=encoder_outputs, labels=batch["labels"])
        else:
            teacher_outputs = teacher_model(**batch)

    # CE (data) loss
    ce_loss = student_outputs.loss

    # log softmax / softmax for numerical stability
    student_distribution = nn.functional.log_softmax(student_outputs.logits, dim=-1)
    teacher_distribution = nn.functional.softmax(teacher_outputs.logits, dim=-1)
    # temperature is always 1 for eval
    kl_loss = kl_divergence(teacher_distribution, student_distribution, batch["labels"])

    # use Distil-Whisper formulation (fix weight of CE loss and tune KL weight)
    loss = 0.8 * ce_loss + training_args.kl_weight * kl_loss
    metrics = {"loss": loss, "ce_loss": ce_loss, "kl_loss": kl_loss}
    return metrics

def generate_step(batch):
    student_model.eval()
    output_ids = accelerator.unwrap_model(student_model).generate(batch["input_features"], **gen_kwargs)
    output_ids = accelerator.pad_across_processes(output_ids, dim=1, pad_index=tokenizer.pad_token_id)
    return output_ids

In [35]:
logger.info("***** Running training *****")
logger.info(f"  Num examples = {total_train_steps * train_batch_size * gradient_accumulation_steps}")
if not data_args.streaming:
    logger.info(f"  Num epochs = {num_epochs}")
logger.info("  Instantaneous batch size per device =" f" {training_args.per_device_train_batch_size}")
logger.info("  Gradient accumulation steps =" f" {gradient_accumulation_steps}")
logger.info(
    f"  Total train batch size (w. parallel & distributed) = {train_batch_size * gradient_accumulation_steps}"
)
logger.info(f"  Total optimization steps = {total_train_steps}")

# ======================== Training ================================
train_time = 0
train_start = time.time()
steps_trained_progress_bar = tqdm(
    range(total_train_steps), desc="Train steps ... ", position=0, disable=not accelerator.is_local_main_process
)
continue_training = True
epochs_trained = 0
cur_step = 0

checkpoint = None
if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint

if checkpoint is not None:
    accelerator.load_state(checkpoint)
    # Find num steps and epoch from saved state string pattern
    pattern = r"checkpoint-(\d+)-epoch-(\d+)"
    match = re.search(pattern, checkpoint)
    cur_step = int(match.group(1))
    epochs_trained = int(match.group(2))

    logger.info("  Continuing training from checkpoint, will skip to saved global_step")
    logger.info(f"  Continuing training from epoch {epochs_trained}")
    logger.info(f"  Continuing training from global step {cur_step}")

    steps_trained_progress_bar.update(cur_step)

    for epoch in range(0, epochs_trained):
        vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)

    if not data_args.streaming and training_args.max_steps < 0:
        # we know exactly the number of steps per epoch, so can skip through the required number of batches
        resume_step = (cur_step - epochs_trained * steps_per_epoch) * gradient_accumulation_steps
    else:
        # Currently we don't know how many steps we've taken in the current epoch
        # So we just shuffle the dataset one extra time and start from a fresh epoch
        # This is "good enough" for our purposes but not fully correct
        resume_step = None
        vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)
else:
    resume_step = None

05/08/2024 19:09:57 - INFO - src.utils - ***** Running training *****
05/08/2024 19:09:58 - INFO - src.utils -   Num examples = 110000
05/08/2024 19:09:58 - INFO - src.utils -   Num epochs = 30
05/08/2024 19:09:58 - INFO - src.utils -   Instantaneous batch size per device = 11
05/08/2024 19:09:58 - INFO - src.utils -   Gradient accumulation steps = 1
05/08/2024 19:09:58 - INFO - src.utils -   Total train batch size (w. parallel & distributed) = 11
05/08/2024 19:09:58 - INFO - src.utils -   Total optimization steps = 10000
Train steps ... :   0%|          | 0/10000 [00:00<?, ?it/s]

In [36]:
for epoch in range(epochs_trained, num_epochs):
    vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)
    train_dataloader = DataLoader(
        vectorized_datasets["train"],
        collate_fn=data_collator,
        batch_size=per_device_train_batch_size,
        num_workers=dataloader_num_workers,
        prefetch_factor=prefetch_factor,
        pin_memory=training_args.dataloader_pin_memory,
    )
    train_dataloader = accelerator.prepare(train_dataloader)
    if hasattr(train_dataloader, "dataset") and isinstance(train_dataloader.dataset, IterableDataset):
        train_dataloader.dataset.set_epoch(epoch)

    if resume_step is not None:
        # Skip the first N batches in the dataloader when resuming from a checkpoint
        train_dataloader = accelerator.skip_first_batches(train_dataloader, resume_step)
        resume_step = None

    for batch in train_dataloader:
        with accelerator.accumulate(student_model):
            loss, train_metric = train_step(batch, temperature=training_args.temperature)
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(student_model.parameters(), training_args.max_grad_norm)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Check if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            steps_trained_progress_bar.update(1)
            cur_step += 1

            if cur_step % training_args.logging_steps == 0:
                steps_trained_progress_bar.write(
                    f"Step... ({cur_step} / {total_train_steps} | Loss:"
                    f" {train_metric['loss']}, Learning Rate:"
                    f" {lr_scheduler.get_last_lr()[0]})"
                )
                log_metric(
                    accelerator,
                    metrics=train_metric,
                    learning_rate=lr_scheduler.get_last_lr()[0],
                    train_time=train_time + time.time() - train_start,
                    step=cur_step,
                    epoch=epoch,
                    prefix="train",
                )

            # save checkpoint and weights after each save_steps and at the end of training
            if (cur_step % training_args.save_steps == 0) or cur_step == total_train_steps:
                intermediate_dir = os.path.join(training_args.output_dir, f"checkpoint-{cur_step}-epoch-{epoch}")
                accelerator.save_state(output_dir=intermediate_dir)
                accelerator.wait_for_everyone()
                if accelerator.is_main_process:
                    rotate_checkpoints(training_args.save_total_limit, output_dir=training_args.output_dir)


            if training_args.do_eval and (cur_step % eval_steps == 0 or cur_step == total_train_steps):
                train_time += time.time() - train_start
                student_model.eval()
                # ======================== Evaluating ==============================
                for eval_split in all_eval_splits:
                    eval_metrics = []
                    eval_preds = []
                    eval_labels = []
                    eval_start = time.time()

                    validation_dataloader = DataLoader(
                        vectorized_datasets[eval_split],
                        collate_fn=data_collator,
                        batch_size=per_device_eval_batch_size,
                        drop_last=False,
                        num_workers=dataloader_num_workers,
                        prefetch_factor=prefetch_factor,
                        pin_memory=training_args.dataloader_pin_memory,
                    )
                    validation_dataloader = accelerator.prepare(validation_dataloader)

                    for batch in tqdm(
                        validation_dataloader,
                        desc=f"Evaluating {eval_split}...",
                        position=2,
                        disable=not accelerator.is_local_main_process,
                    ):
                        # Model forward
                        eval_metric = eval_step(batch)
                        eval_metric = accelerator.gather_for_metrics(eval_metric)
                        eval_metrics.append(eval_metric)

                        # generation
                        if training_args.predict_with_generate:
                            generated_ids = generate_step(batch)
                            # Gather all predictions and targets
                            generated_ids, labels = accelerator.gather_for_metrics(
                                (generated_ids, batch["labels"])
                            )
                            eval_preds.extend(generated_ids)
                            eval_labels.extend(labels)

                    eval_time = time.time() - eval_start
                    # normalize eval metrics
                    eval_metrics = {
                        key: torch.mean(torch.stack([d[key] for d in eval_metrics])) for key in eval_metrics[0]
                    }

                    # compute WER metric
                    wer_desc = ""
                    if training_args.predict_with_generate:
                        wer_metric, pred_str, label_str, norm_pred_str, norm_label_str = compute_metrics(
                            eval_preds, eval_labels
                        )
                        eval_metrics.update(wer_metric)
                        wer_desc = " ".join([f"Eval {key}: {value} |" for key, value in wer_metric.items()])
                        log_pred(
                            accelerator,
                            pred_str,
                            label_str,
                            norm_pred_str,
                            norm_label_str,
                            step=cur_step,
                            prefix=eval_split,
                        )

                    # Print metrics and update progress bar
                    steps_trained_progress_bar.write(
                        f"Eval results for step ({cur_step} / {total_train_steps} | Eval Loss: {eval_metrics['loss']} |"
                        f" {wer_desc})"
                    )

                    log_metric(
                        accelerator,
                        metrics=eval_metrics,
                        train_time=eval_time,
                        step=cur_step,
                        epoch=epoch,
                        prefix=eval_split,
                    )

                # flush the train metrics
                train_start = time.time()

            # break condition
            if cur_step == total_train_steps:

                # un-wrap student model for save
                student_model = accelerator.unwrap_model(student_model)
                student_model.save_pretrained(training_args.output_dir)

                continue_training = False
                break

    if not continue_training:
        break

accelerator.end_training()

c:\Users\Zhenya\anaconda3\envs\environment\lib\site-packages\transformers\models\whisper\modeling_whisper.py:694: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
Train steps ... :   0%|          | 25/10000 [00:26<1:40:06,  1.66it/s]

Step... (25 / 10000 | Loss: 10.739805221557617, Learning Rate: 3.4000000000000007e-05)


Train steps ... :   0%|          | 50/10000 [00:41<1:40:39,  1.65it/s]

Step... (50 / 10000 | Loss: 7.542468070983887, Learning Rate: 8.4e-05)


Train steps ... :   1%|          | 75/10000 [00:56<1:40:54,  1.64it/s]

...

Step... (10000 / 10000 | Loss: 0.3369992971420288, Learning Rate: 0.0001)


05/08/2024 22:47:48 - INFO - accelerate.checkpointing - Model weights saved in models\distiled_student_model_moz_v1\checkpoint-10000-epoch-29\model.safetensors
05/08/2024 22:47:48 - WARNING - accelerate.utils.other - Removed shared tensor {'proj_out.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
05/08/2024 22:47:49 - INFO - accelerate.checkpointing - Model weights saved in models\distiled_student_model_moz_v1\checkpoint-10000-epoch-29\model_1.safetensors
05/08/2024 22:47:50 - INFO - accelerate.checkpointing - Optimizer state saved in models\distiled_student_model_moz_v1\checkpoint-10000-epoch-29\optimizer.bin
05/08/2024 22:47:50 - INFO - accelerate.checkpointing - Scheduler state saved in models\distiled_student_model_moz_v1\checkpoint-10000-epoch-29\scheduler.bin
05/08/2024 22:47:50 - INFO - accelerate.checkpointing - Sampler state for dataloader 0 saved in models\distiled_student_model_moz_v1\checkpoint-10000-epoch-

Eval results for step (10000 / 10000 | Eval Loss: 0.9954593777656555 | Eval wer: 23.274271489237076 | Eval wer_ortho: 33.57207838153962 |)


Model weights saved in ./models/distiled_student_model_moz_v1\model.safetensors


eval/ce_loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
eval/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▇▇▇██
eval/kl_loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
eval/loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
eval/time,▁▆▄▃▃▄▂▂▂▃▁▂▃▂▃▂█▃▄▃
eval/wer,█▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
eval/wer_ortho,█▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train/ce_loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/kl_loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [37]:
!nvidia-smi

Wed May  8 22:53:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
| 73%   55C    P8             35W /  350W |   20648MiB /  24576MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [38]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


In [39]:
Train steps ... :   2%|▎         | 25/1000 [00:38<11:11,  1.45it/s]  
Step... (25 / 1000 | Loss: 10.911069869995117, Learning Rate: 3.4000000000000007e-05)
Train steps ... :   5%|▌         | 50/1000 [01:15<20:51,  1.32s/it]  
Step... (50 / 1000 | Loss: 7.235431671142578, Learning Rate: 8.4e-05)
Train steps ... :   8%|▊         | 75/1000 [01:40<11:14,  1.37it/s]
Step... (75 / 1000 | Loss: 6.183444023132324, Learning Rate: 0.0001)
Train steps ... :  10%|█         | 100/1000 [02:14<10:15,  1.46it/s]
Step... (100 / 1000 | Loss: 4.9677629470825195, Learning Rate: 0.0001)
Train steps ... :  12%|█▎        | 125/1000 [02:50<19:15,  1.32s/it]

SyntaxError: invalid character '▎' (U+258E) (1250305899.py, line 1)